In [49]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output, State, ALL, MATCH
from dash import callback_context as ctx
import dash_bootstrap_components as dbc
import plotly.express as px
import json

df = pd.read_csv('data/train.csv')

columns = list(df.columns)
numeric_column = ['bant_submit', 'historical_existing_cnt', 'lead_desc_length']
target_column = [columns[-1]]
categorical_column = [col for col in columns[:-1] if col not in numeric_column]
column_dict = {'target': target_column, 'categorical': categorical_column, 'numeric': numeric_column}
column_discription = {'business_unit': '사업부', 'customer_idx': '회사 고유 Index', 'customer_type': '고객 유형',
                      'enterprise': '회사 규모', 'customer_job': '고객 업종', 'inquiry_type': '문의 유형',
                      'product_category': '제품 대분류', 'product_subcategory': '제품 중분류', 'product_modelname': '제품 모델명',
                      'customer_position': '고객 직책', 'response_corporate': '담당 법인', 'expected_timeline': '희망 구매 날짜',
                      'business_area': '사업 도메인', 'business_subarea': '사업 세부 도메인', 'lead_owner': '영업 담당자',
                      'bant_submit': 'MQL 정보의 BANT 가중치', 'historical_existing_cnt': '과거 영업 전환 횟수', 'lead_desc_length': '고객 요청사항 글자 수',
                      'is_converted': '영업 전환 여부'}

In [50]:
def generate_table(column_type):
    choosed_columns = column_dict[column_type]
    column_contents = ['Column_Name', 'Description', 'Dtype', 'Null_Percentage']

    table = html.Table([
                        html.Thead(
                            html.Tr([html.Th(col) for col in column_contents])
                        ),
                        html.Tbody([
                            html.Tr([html.Td(s) for s in [
                                dbc.Button(col, id={"type": "button", "id": col}), column_discription[col], str(df[col].dtype), round(df.isna().sum()[col]/len(df), 2)
                            ]]) for col in choosed_columns])
                        ])
    
    return table

def generate_graph(selected_column, selected_column_type):

    def target_graph(selected_column):
        pass

    def categorical_graph(selected_column):
        pass

    def numeric_graph(selected_column):
        pass

    graph_type = {'target': target_graph, 'categorical': categorical_graph, 'numeric': numeric_graph}

    return graph_type[selected_column_type](selected_column)

In [52]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

column_types = ['target', 'categorical', 'numeric']

tabs = html.Div([
    dbc.Tabs(
        [dbc.Tab(label=col_type.title(), tab_id=col_type) for col_type in column_types],
        id='tabs',
        active_tab=column_types[0],
    ),
    html.Div([html.Table(id='content')]),
], style={'width': '49%'})

graph = html.Div([
       dcc.Store(id='selected_column'), html.Span(id='fig')
    ], style={'width': '49%', 'float': 'right'})

app.layout=html.Div([tabs,
                     graph,
                     ])


@app.callback(
    Output('content', 'children'),
    Input('tabs', 'active_tab')
)
def display_table(column_type):
    return generate_table(column_type)


@app.callback(
    Output('selected_column', 'data'),
    Input({"type": "button", "id": ALL}, "n_clicks"),
)
def get_triggered_column(clicks):
    if not any(clicks):
        return None
    
    triggered_id = ctx.triggered[0]["prop_id"].split(".")[0]
    triggered_column = json.loads(triggered_id)['id']

    return triggered_column


@app.callback(
    Output('fig', 'children'),
    Input('selected_column', 'data'),
    Input('tabs', 'active_tab'),
)
def display_graph(selected_column, selected_column_type):
    if selected_column == None:
        return 'please select a column'
    
    return f'you choosed {selected_column} from {selected_column_type} column'


# @app.callback(
#     Output('fig', 'children'),
#     Input('selected_column', 'data'),
#     Input('tabs', 'active_tab'),
# )
# def display_graph(selected_column, selected_column_type):
#     return generate_graph(selected_column, selected_column_type)

app.run(debug=True)